In [1]:
import os
import dotenv

In [2]:
loaded = dotenv.load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [3]:
from langchain_community.document_loaders import PyPDFLoader

d:\tools\envs\my_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%pwd

'c:\\Users\\Anas Fareedi\\Desktop\\interview_que_ans\\notebook'

In [5]:
%cd ..


c:\Users\Anas Fareedi\Desktop\interview_que_ans


C:\Users\Anas Fareedi\AppData\Roaming\Python\Python310\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
file_path = "static/input/SDG.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [7]:
len(data)

24

In [8]:
from langchain_text_splitters import TokenTextSplitter

In [9]:
question_gen = ""
for page in data:
    question_gen += page.page_content

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter_ques_gen = RecursiveCharacterTextSplitter(
    chunk_size=10000, 
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

In [11]:
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)

In [12]:
len(chunk_ques_gen)

2

In [13]:
from langchain_core.documents import Document

In [14]:
document_ques_gen = [Document(page_content = t) for t in chunk_ques_gen]

In [15]:
splitter_ans_gen = RecursiveCharacterTextSplitter(
    chunk_size=10000, 
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

In [16]:
document_answer_gen = splitter_ans_gen.split_documents(
    document_ques_gen
)

In [17]:
len(document_answer_gen)

2

In [73]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [74]:
llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-3-flash-preview")

In [75]:
from langchain_core.prompts.prompt import PromptTemplate

In [76]:
text = loader.load()[0].page_content

In [77]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:
i want you to generate total 5 question only it is very important
------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [78]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [79]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.

------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)


In [80]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [81]:
from langchain_classic.chains.summarize import load_summarize_chain

In [82]:
from langchain_core.output_parsers import StrOutputParser

# Create a chain that processes documents
chain = PROMPT_QUESTIONS | llm | StrOutputParser()


In [83]:
ques = chain.invoke(document_ques_gen[0].page_content)

print(ques)

To prepare you for technical assessments or domain-specific coding tests related to social impact data and the Sustainable Development Goals (SDGs), here are 5 targeted questions based on the provided documentation:

**1. Data Structure and Initialization:**
Based on the introductory text, if you were to initialize a global tracking system for the Sustainable Development Goals, what is the specific integer count of participating countries, the total number of goals (SDGs) defined, and the exact 15-year timeframe (start and end years) established for the plan?

**2. Threshold Logic and Metrics (Goal 1):**
In developing an algorithm to identify individuals living in "extreme poverty" as defined by Goal 1, what specific daily income threshold (in USD) would serve as your Boolean trigger, and how many people globally currently fall below this value?

**3. Quantitative Analysis of Infrastructure (Goal 9):**
Goal 9 focuses on bridging the "digital divide." According to the documentation, wha

In [84]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [85]:
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [86]:
from langchain_community.vectorstores import FAISS

In [87]:
vector_store = FAISS.from_documents(document_answer_gen, embedding)

In [88]:
llm_answer_gen = ChatGoogleGenerativeAI(temperature=0.1, model="gemini-3-flash-preview")

In [89]:
ques

'To prepare you for technical assessments or domain-specific coding tests related to social impact data and the Sustainable Development Goals (SDGs), here are 5 targeted questions based on the provided documentation:\n\n**1. Data Structure and Initialization:**\nBased on the introductory text, if you were to initialize a global tracking system for the Sustainable Development Goals, what is the specific integer count of participating countries, the total number of goals (SDGs) defined, and the exact 15-year timeframe (start and end years) established for the plan?\n\n**2. Threshold Logic and Metrics (Goal 1):**\nIn developing an algorithm to identify individuals living in "extreme poverty" as defined by Goal 1, what specific daily income threshold (in USD) would serve as your Boolean trigger, and how many people globally currently fall below this value?\n\n**3. Quantitative Analysis of Infrastructure (Goal 9):**\nGoal 9 focuses on bridging the "digital divide." According to the document

In [90]:
ques_list = ques.split("\n")

In [91]:
ques_list

['To prepare you for technical assessments or domain-specific coding tests related to social impact data and the Sustainable Development Goals (SDGs), here are 5 targeted questions based on the provided documentation:',
 '',
 '**1. Data Structure and Initialization:**',
 'Based on the introductory text, if you were to initialize a global tracking system for the Sustainable Development Goals, what is the specific integer count of participating countries, the total number of goals (SDGs) defined, and the exact 15-year timeframe (start and end years) established for the plan?',
 '',
 '**2. Threshold Logic and Metrics (Goal 1):**',
 'In developing an algorithm to identify individuals living in "extreme poverty" as defined by Goal 1, what specific daily income threshold (in USD) would serve as your Boolean trigger, and how many people globally currently fall below this value?',
 '',
 '**3. Quantitative Analysis of Infrastructure (Goal 9):**',
 'Goal 9 focuses on bridging the "digital divide

In [92]:
# from langchain_classic.chains.qa import retrieval_qa
from langchain_classic.chains import RetrievalQA

In [93]:
answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen, 
                                               chain_type="stuff", 
                                               retriever=vector_store.as_retriever())


In [ ]:
# Answer each question and save to a file
for question in ques_list:
    question = question.strip()
    if not question:
        continue
    print("Question: ", question)
    result = answer_generation_chain.invoke(question)
    answer_text = result.get("result") if isinstance(result, dict) else str(result)
    print("Answer: ", answer_text)
    print("--------------------------------------------------\n\n")
    with open("answers.txt", "a", encoding="utf-8") as f:
        f.write("Question: " + question + "\n")
        f.write("Answer: " + answer_text + "\n")
        f.write("--------------------------------------------------\n\n")
